# 6. Sieci neuronowe w PyTorch

### Tomasz Rodak

Laboratorium 6

---

## 

## 6.1 

Celem tego arkusza jest wprowadzenie do sieci neuronowych w [PyTorch](https://pytorch.org/). 

[**Sieć neuronowa**](https://en.wikipedia.org/wiki/Neural_network_(machine_learning)) to ogólny termin oznaczający rodzinę modeli o strukturze inspirowanej budową mózgu. Proste sieci neuronowe mają strukturę jednokierunkowego grafu: dane wejściowe przechodzą przez kolejne węzły (warstwy neuronów) wzdłuż krawędzi (połączeń neuronów). Przy przejściu przez krawędź wartość jest mnożona przez wagę krawędzi, w węźle następuje agregacja wartości z krawędzi, a następnie zastosowanie funkcji aktywacji do wyniku tej agregacji.

**PyTorch** to operująca na tensorach biblioteka do głębokiego uczenia, która jest zoptymalizowana pod kątem wydajności na CPU i procesorach graficznych GPU.

Poniżej przedstawimy demonstrację działania prostej sieci neuronowej przeznaczonej do wykonywania klasyfikacji:
- utworzymy syntetyczny zbiór danych w postaci punktów w przestrzeni 2D (dzięki czemu będziemy mogli wizualizować wyniki),
- zbudujemy sieć neuronową,
- pokażemy jak wykonać jej trening,
- zwizualizujemy wyniki klasyfikacji.

### 6.1.2 Importowanie bibliotek

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, accuracy_score

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

from torchinfo import summary

### 6.1.3 Generowanie danych

In [ ]:
N = 5_000
r1, r2 = 1, 2
sd = .2
t = np.linspace(0, 2 * np.pi, N//2)
X = np.zeros((N, 2))
X[:N//2, 0] = r1 * np.cos(t) + np.random.normal(0, sd, N//2)
X[:N//2, 1] = r1 * np.sin(t) + np.random.normal(0, sd, N//2)
X[N//2:, 0] = r2 * np.cos(t) + np.random.normal(0, sd, N//2)
X[N//2:, 1] = r2 * np.sin(t) + np.random.normal(0, sd, N//2)
y = np.zeros(N)
y[N//2:] = 1
plt.scatter(X[:, 0], X[:, 1], c=y, s=5, alpha=0.5);

### 6.1.4 Konwersja danych do PyTorch

Aby PyTorch mógł wykonać obliczenia na danych, muszą one mieć format **tensora**. Tensory to wielowymiarowe tablice numeryczne, które są podstawowym typem danych w PyTorch. W porównaniu do tablic NumPy tensory mają dodatkowe właściwości, takie jak możliwość wykonywania obliczeń na GPU, automatyczne różniczkowanie i możliwość śledzenia gradientów.

Funkcja `torch.tensor()` konwertuje tablicę NumPy na tensor PyTorch. Korzystamy ponadto z:
- `torch.utils.data.TensorDataset` - klasa, która tworzy zbiór danych z tensorów,
- `torch.utils.data.DataLoader` - klasa, która tworzy iteratory na zbiorach danych, dzieląc je na partie zwane batchami (*ang. batch*). 

In [ ]:
data = torch.tensor(X, dtype=torch.float32)
labels = torch.tensor(y, dtype=torch.float32)

train_dataset = TensorDataset(data, labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

### 6.1.5 Budowanie modelu

W PyTorch najczęściej definiuje się architekturę sieci neuronowej, tworząc klasę dziedziczącą po `torch.nn.Module`. W tej klasie:

- **Konstruktor `__init__`:** Inicjalizuje się tutaj wszystkie warstwy sieci, takie jak warstwy liniowe (`nn.Linear`), konwolucyjne (`nn.Conv2d`), itd.
- **Metoda `forward`:** Określa się przebieg danych przez sieć, czyli w jaki sposób kolejne warstwy przetwarzają dane wejściowe.

W podanym niżej przykładzie:
- W metodzie `__init__` definiujemy dwie warstwy liniowe.
- W metodzie `forward` przekazujemy dane przez warstwę wejściową, stosujemy funkcję aktywacji ReLU oraz przechodzimy przez kolejną warstwę.

In [ ]:
class Klasyfikator(nn.Module):
    
    def __init__(self):
        super(Klasyfikator, self).__init__()
        # Warstwa liniowa: 2 wejścia (wymiary danych), 30 neuronów w warstwie ukrytej
        self.fc1 = nn.Linear(2, 4)
        # Warstwa liniowa: 30 neuronów w warstwie ukrytej, 10 neuronów w kolejnej warstwie
        self.fc2 = nn.Linear(4, 2)
        # Warstwa liniowa: 10 neuronów w warstwie ukrytej, 1 wyjście (prawdopodobieństwo klasy)
        self.fc3 = nn.Linear(2, 1)
        # Funkcja aktywacji ReLU
        self.relu = nn.ReLU()
        # Funkcja aktywacji Sigmoid (do klasyfikacji binarnej)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Przejście przez pierwszą warstwę liniową i zastosowanie ReLU
        x = self.fc1(x)
        x = self.relu(x)
        # Przejście przez drugą warstwę liniową i zastosowanie ReLU
        x = self.fc2(x)
        x = self.relu(x)
        # Przejście przez trzecią warstwę liniową i zastosowanie Sigmoid
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

Utworzenie obiektu klasy `Klasyfikator` - jest to sieć neuronowa zbudowana zgodnie z szablonem zdefiniowanym w klasie `Klasyfikator`:

In [ ]:
clf = Klasyfikator()

Przepchnięcie jednej próbki przez sieć neuronową:

In [ ]:
clf.forward(torch.tensor([[1.0, 2.0]]))  # Przykładowe wejście

Tak klasyfikator sam się przedstawia:

In [ ]:
clf

Podsumowanie wykonane przez funkcję `torchinfo.summary()`:

In [ ]:
summary(clf, input_size=(1, 2), col_names=["input_size", "output_size", "num_params"])

Obliczenie przewidywań dla wszystkich próbek w zbiorze treningowym (jeszcze przed rozpoczęciem treningu):

In [ ]:
clf.eval() # Ustawienie modelu w tryb ewaluacji
y_proba = clf.forward(train_dataset[:][0])
y_pred = y_proba > 0.5

Zmienna `y_proba` to tensor zawierający prawdopodobieństwa przynależności do klas. My mamy tylko dwie klasy a warstwie wyjściowej ustawiliśmy funkcję sigmoid na jednowymiarowym wyjściu, więc `y_proba` to tensor jednowymiarowy z prawdopodobieństwami przynależności do klasy 1. Tensor `y_pred` zamienia prawdopodobieństwa na klasy (0 lub 1) względem progu 0.5. 

Aktualna ocena modelu na zbiorze treningowym:

In [ ]:
accuracy_score(y, y_pred.detach().numpy())  # Obliczenie dokładności modelu

In [ ]:
confusion_matrix(y, y_pred.detach().numpy())  # Macierz pomyłek

Wizualizacja przewidywań przed rozpoczęciem treningu:

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_pred.detach().numpy(), s=1, alpha=0.5)

### 6.1.6 Funkcja straty

Funkcja straty to miara różnicy między przewidywaniami modelu a rzeczywistymi wartościami. W przypadku klasyfikacji binarnej najczęściej stosuje się funkcję straty `Binary Cross-Entropy Loss`, która jest dostępna w PyTorch jako `torch.nn.BCELoss`. Funkcja ta oblicza stratę na podstawie prawdopodobieństw przynależności do klasy 1.

In [ ]:
loss_fn = nn.BCELoss()

Wartość straty przed treningiem:

In [ ]:
loss_fn(y_proba, train_dataset[:][1].unsqueeze(1))  # Obliczenie straty

### 6.1.7 Optymalizator

Optymalizator to algorytm, który aktualizuje wagi modelu w celu minimalizacji funkcji straty. W PyTorch dostępnych jest wiele optymalizatorów, takich jak `SGD`, `Adam`, `RMSprop`, itd. W naszym przypadku użyjemy optymalizatora `Adam`, który jest dostępny w PyTorch jako `torch.optim.Adam`. Parametr `lr` to współczynnik uczenia, który kontroluje szybkość aktualizacji wag. 

In [ ]:
optimizer = torch.optim.Adam(clf.parameters(), lr=0.01)

### 6.1.8 Trening modelu

Trening modelu polega na wielokrotnym przechodzeniu przez zbiór danych, obliczaniu straty i aktualizowaniu wag modelu. **Iteracją** nazywamy przetworzenie jednego batcha danych - następuje wtedy obliczenie straty i aktualizacja wag. **Epoką** nazywamy przetworzenie całego zbioru danych, czyli przejście przez wszystkie batche, na które dzielimy zbiór danych. Ważna uwaga: po każdej epoce zbiór danych jest mieszany, dlatego odpowiadające sobie batch'e w kolejnych epokach zawierają różne próbki.

In [ ]:
n_epochs = 4  # Liczba epok treningowych

clf.train()  # Ustawienie modelu w tryb treningu

# Pętla treningowa
for epoch in range(n_epochs):
    for x_batch, y_batch in train_loader:  # Iteracja po batchach danych
        optimizer.zero_grad()  # Wyzerowanie gradientów
        y_proba = clf(x_batch)  # Obliczenie przewidywań modelu
        loss = loss_fn(y_proba.flatten(), y_batch)  # Obliczenie straty
        loss.backward()  # Obliczenie gradientów
        optimizer.step()  # Aktualizacja wag modelu
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}")  # Wyświetlenie straty z ostaniej iteracji

Przewidywania modelu na zbiorze treningowym:

In [ ]:
clf.eval()
y_proba = clf.forward(train_dataset[:][0])
y_pred = y_proba > 0.5

In [ ]:
accuracy_score(y, y_pred.detach().numpy())  # Obliczenie dokładności modelu

In [ ]:
confusion_matrix(y, y_pred.detach().numpy())  # Macierz pomyłek

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_pred.detach().numpy(), s=5, alpha=0.5, cmap='coolwarm')

## 6.2 Zadanie

Wykonaj eksperymenty zmieniając:
- liczbę neuronów w warstwie ukrytej,
- liczbę warstw ukrytych,
- funkcję aktywacji,
- współczynnik uczenia,
- liczbę epok,
- rozmiar batcha,
- rozmiar zbioru treningowego,
- postać zbioru treningowego (np. zmień odchylenie standardowe `sd` generowanych punktów)